In [1]:
!zenml integration install sklearn -y

⠙ Installing integrations.....


In [2]:
# To restart my Kernel

import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True) 

{'status': 'ok', 'restart': True}

In [9]:
NGROK_TOKEN = "2qPpzY9EnO7Ju0x9NnaPno7fuOK_3jh2FhYiFMti2E9suW2y9"

!ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /home/ammar/snap/ngrok/226/.config/ngrok/ngrok.yml


In [10]:
from zenml.environment import Environment


In [12]:
# for ZenML Setup

!rm -rf .zen
!zenml init

Initializing the ZenML global configuration version to 0.71.0
⠋ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/project1.
Creating database tables
⠙ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/project1.
⠹ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/project1.
⠼ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/project1.
⠴ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/project1.
⠦ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/project1.
⠇ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/project1.
⠏ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/project1.
⠋ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/project1.
⠙ Initializing ZenML repository at /media/ammar/Volume 
2/projects/Projects/MLOps/pro

In [16]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [17]:
def train_test() -> None:
    """
        train and test a Scikit Learn SVC Classifier on digits
    """

    digits = load_digits()
    image_shape = (len(digits.images), -1) 
    data = digits.images.reshape(image_shape)

    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=True
    )

    mode = SVC(gamma=0.001)
    mode.fit(X_train, y_train)
    test_acc = mode.score(X_train, y_train)
    print(f"Model Accuracy is : {test_acc}")

In [18]:
train_test()

Model Accuracy is : 0.9986082115518441


In [33]:
from zenml import step
import pandas as pd
import numpy as np
from typing import Tuple
from typing_extensions import Annotated

In [36]:
@step
def split_data() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    """
    Loads the digits dataset, reshapes the images, splits the data into training and test sets.
    
    Returns:
        X_train: Training data features.
        X_test: Test data features.
        y_train: Training data labels.
        y_test: Test data labels.
    """
    digits = load_digits()
    image_shape = (len(digits.images), -1)
    data = digits.images.reshape(image_shape)

    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=True
    )
    
    return X_train, X_test, y_train, y_test


@step
def model_trainer(X_train: np.ndarray, y_train: np.ndarray) -> ClassifierMixin:
    """
    Trains a Support Vector Classifier (SVC) model on the provided training data.

    Args:
        X_train: Training data features.
        y_train: Training data labels.

    Returns:
        model: Trained SVC model.
    """
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)

    return model


@step
def evaluator(model: ClassifierMixin, X_test: np.ndarray, y_test: np.ndarray) -> float:
    """
    Evaluates the accuracy of the trained model on the test data.

    Args:
        model: The trained model to evaluate.
        X_test: Test data features.
        y_test: Test data labels.

    Returns:
        test_acc: Accuracy of the model on the test set.
    """
    test_acc = model.score(X_test, y_test)
    print(f"Model Accuracy is: {test_acc}")
    
    return test_acc


In [37]:
from zenml import pipeline

In [49]:
@pipeline
def digital_pipeline():
    """
    Defines a ZenML pipeline for loading data, training a model, and evaluating its performance.
    
    Steps:
        1. Loads the digits dataset and splits it into training and test sets.
        2. Trains a Support Vector Classifier (SVC) model using the training data.
        3. Evaluates the model's accuracy on the test data.
    """
    X_train, X_test, y_train, y_test = split_data()
    model = model_trainer(X_train=X_train, y_train=y_train)
    evaluator(model=model, X_test=X_test, y_test=y_test)


In [50]:
digital_model_pipeline = digital_pipeline()
# digital_model_pipeline.run(unlisted=True)

Initiating a new run for the pipeline: digital_pipeline.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Using cached version of step split_data.
Using cached version of step model_trainer.
Using cached version of step evaluator.
All steps of the pipeline run were cached.


In [51]:
# !zenml login --local
from zenml.environment import Environment
from pyngrok import ngrok


NGROK_TOKEN = "2qPpzY9EnO7Ju0x9NnaPno7fuOK_3jh2FhYiFMti2E9suW2y9"

!ngrok authtoken {NGROK_TOKEN}

def start_zenml_dashboard(port=8237):
    try:
        public_url = ngrok.connect(port)
        print(f"Dashboard URL is : {public_url}")
    
        !zenml up --port {port}
    except:
        !zenml up --port {port}

start_zenml_dashboard()

Authtoken saved to configuration file: /home/ammar/.config/ngrok/ngrok.yml
Opening tunnel named: http-8237-b7c3c73a-fa97-4195-a8f1-a4a31cae5f0a
Dashboard URL is : NgrokTunnel: "https://ce45-156-197-228-254.ngrok-free.app" -> "http://localhost:8237"
The `zenml up` command is deprecated and will be removed in a future release. 
Please use the `zenml login --local` command instead.
Calling `zenml login --local`...
The local daemon ZenML server is already configured with the same parameters.
⠋ Starting service 'DaemonZenServer[79c078fb-a55e-47bd-ab23-1cffdebb196f] (type:
zen_server, flavor: daemon)'.

Connecting to the local daemon ZenML server (http://127.0.0.1:8237).
Updated the global store configuration.
Connected to the local daemon ZenML server (http://127.0.0.1:8237).
The local ZenML dashboard is available at 'http://127.0.0.1:8237'.
Automatically opening the dashboard in your browser. To disable this, set the env variable AUTO_OPEN_DASHBOARD=false.
Gtk-Message: 23:17:24.730: Not lo